<img src="https://www.rp.edu.sg/images/default-source/default-album/rp-logo.png" width="200" alt="Republic Polytechnic"/>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/koayst-rplesson/SST_DP2025/blob/main/L10/L10.ipynb)

# Setup and Installation

You can run this Jupyter notebook either on your local machine or run it at Google Colab.

* For local machine, it is recommended to install Anaconda and create a new development environment called `SST_DP2025`.
* Pip/Conda install the libraries stated below when necessary.
---

# <font color='red'>ATTENTION</font>

## Google Colab
- If you are running this code in Google Colab, **DO NOT** store the API Key in a text file and load the key later from Google Drive. This is insecure and will expose the key.
- **DO NOT** hard code the API Key directly in the Python code, even though it might seem convenient for quick development.
- You need to enter the API key at python code `getpass.getpass()` when ask.

## Local Environment/Laptop
- If you are running this code locally in your laptop, you can create a env.txt and store the API key there.
- Make sure env.txt is in the same directory of this Jupyter notebook.
- You need to install `python-dotenv` and run the Python code to load in the API key.

---
```
%pip install python-dotenv

from dotenv import load_dotenv

load_dotenv('env.tx')
openai_api_key = os.getenv('OPENAI_API_KEY')
```
---

## GitHub/GitLab
- **DO NOT** `commit` or `push` API Key to services like GitHub or GitLab.



# Lesson 10

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain
%pip install --quiet -U langgraph
%pip install --quiet -U langchain-openai
%pip install --quiet -U grandalf

In [ ]:
# grandalf         0.8
# langchain        0.3.11
# langgraph        0.2.59
# langchain-core   0.3.24
# langchain-openai 0.2.12
# openai           1.57.2
# pydantic         2.10.3

In [ ]:
import getpass
import os

# setup the OpenAI API Key

# get OpenAI API key ready and enter it when ask
os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Chains

### Basic/Entry Chain
A basic chain or entry chain using LCEL.

In [ ]:
# load langchain libraries

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "How do I say 'Hello' in {language}?",
) 

In [ ]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0.5,
)

In [ ]:
print(model.model_name)

In [ ]:
# TODO: prompt -> model -> StrOutputParser
chain = _____ | _____ | StrOutputParser()

In [ ]:
response = chain.invoke({"language" : "French"})

print(response)

### Simple Sequential Chain (Single Input / Single Output)
Simple chain where the output of one step feed directly into next. A streamlined version of a sequential chain, where the output of each step directly becomes the input for the next. It is perfect for straightforward workflows without the need for advanced memory or branching logic.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from typing import Dict

In [ ]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0.5,
)

In [ ]:
# TODO: create prompt templates

first_prompt = _____.from_template(
    "What is a good name for a company that specialises in making {product}?"
)

second_prompt = ChatPromptTemplate._____(
    "Generate a short introduction of company: {company} ."
)

In [ ]:
chain = (
    first_prompt
    | model
    | {'company' : RunnablePassthrough()}   # 'company' is generated after the first prompt
    | second_prompt
    | model
    | StrOutputParser()
).with_types(input_type=Dict[str, str], output_type=str)

In [ ]:
# TODO: invoke the chain by passing in 'Running Shoe' as product
chain.invoke({'_____' : 'Running shoe'})

In [ ]:
# display the chain as a graph

chain.get_graph().print_ascii()

### Sequential Chain
The `Sequential Chain` in LangChain is a simple way to execute a series of tasks in order, where the output of one step becomes the input for the next. This type of chain is ideal for workflows where the steps depend on one another in a linear sequence.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# TODO: create a chat prompt template from teamplte
first_prompt = _____._____(
    "Write a brief introduction about {topic}."
)

# TODO: first promppt -> model
first_chain = _____ | _____ 

In [ ]:
# TODO: create a chat prompt template from teamplte
second_prompt = _____._____(
    "From this introduction: \"{introduction}\", generate three key bullet points."
)

# TODO: second prompt -> model
second_chain = _____ | _____

In [ ]:
# observe in this prompt, it takes in two inputs 'introduction' and 'bullet_points' from earlier chains

third_prompt = ChatPromptTemplate.from_template(
    "Combine the introduction: \"{introduction}\" with the key points: \"{bullet_points}\".\n"
    "Write a concise summary from this information."
)

# TODO: third prompt -> model
third_chain = third_prompt | model

In [ ]:
chain = (
    {"introduction" : first_chain}
    | RunnablePassthrough.assign(bullet_points=second_chain)
    | RunnablePassthrough.assign(summary=third_chain)
)

In [ ]:
response = chain.invoke({"topic" : "Artificial Intelligence"})

In [ ]:
print(response['summary'].content)

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
# Inspecting Runnables

chain.get_prompts()

### Routing between Sub-Chains
This can be another form of router chain implementation.

In [ ]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
prompt = PromptTemplate.from_template(
"""Given the user question below, classify it as either being about `Math`, `Science`, or `General`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
)

In [ ]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
)

In [ ]:
chain = prompt | model | StrOutputParser()

In [ ]:
# test and ensure the classification is working

chain.invoke({"question" : "What is the basic theory of quantum physics?"})

In [ ]:
# create a math chain

math_chain = PromptTemplate.from_template(
"""You are an expert in Mathematics.
Always answer questions starting with "As my MATH teacher told me".
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model = "gpt-4o-mini")

In [ ]:
# create a science chain

science_chain = PromptTemplate.from_template(
"""You are an expert in Science.
Always answer questions starting with "As my SCIENCE teacher told me".
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model = "gpt-4o-mini")

In [ ]:
# create a generic chain

general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model = "gpt-4o-mini")

In [ ]:
# define a custom function to route between different outputs

# TODO: perform the routing according to info
def route(info):
    if "math" in info['topic'].lower():
        return _____
    elif "science" in info['topic'].lower():
        return _____
    else:
        return _____

In [ ]:
# create the full chain

full_chain = (
    {"topic" : chain, "question" : lambda x: x['question']} 
    | RunnableLambda (route)
)

In [ ]:
# TODO: invoke the chain
response = full_chain._____({"question" : "What is the basic theory of quantum physics?"})

print(response.content)

In [ ]:
# TODO: invoke the chain
response = full_chain.invoke({"_____" : "What is Merlion?"})

print(response.content)

In [ ]:
full_chain.get_prompts()

## Runnable Interface
This `Runnable` interface provides a standard way to create modular and resuable components in a chain or pipeline. It defines the behavior of components that can process inputs and produce outputs, ensuring interoperability across LangChain's ecosystem

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(
    model = 'gpt-4o-mini',
)

In [ ]:
first_prompt = ChatPromptTemplate.from_template("""
You are an expert AI tweet generator. 
Observe the user's prompt, and generate a witty tweet, and include emojis and hashtags.

Prompt: {prompt}
Tweet: """
)

- A simple `tweet_generator` is composed by chaining together a prompt, model and an output parser in sequence.
- Chaining is possible because the components (prompt, model and output parser) implemented the `runnable` interface.

In [ ]:
# TODO: create the chainining: first prompt -> model -> StrOutputParser
tweet_generator = _____ | _____ | _____()

# TODO: invoke tweet generator chain
tweet = tweet_generator._____({"_____": "Langchain releases LangGraph for building stateful, multi-action applications. https://langchain.com"})

In [ ]:
# TODO: fill in the blanks
second_prompt = _____._____("""
You are an expert in AI tweet fixer.
Fix user's original tweet and based it on the mood.

Original Tweet: {tweet}
Mood: {mood}
Fixed Tweet:"""
)

In [ ]:
tweet_fixer = second_prompt | model | StrOutputParser()
fixed_tweet = tweet_fixer.invoke({"tweet" : tweet, "mood" : "funny"})

In [ ]:
fixed_tweet

### RunnableParallel
Execute more than one `Runnable` components concurrently. It is particularly useful for tasks that can be executed independently, as it improves efficiency by running these tasks in parallel.

In [ ]:
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model

In [ ]:
chain2 = ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}") | model

In [ ]:
from langchain_core.runnables.base import RunnableParallel

# TODO: initialise the runnable parallel
parallel_chain = _____(joke=chain1, short_poem=chain2)

In [ ]:
%time

# TODO: invoke the parallel chain
response = parallel_chain._____({"topic" : "bears"})

# the time measured is too tiny to tell the difference between parallel or batch chain

In [ ]:
response

In [ ]:
response['joke'].content

In [ ]:
response['short_poem'].content

In [ ]:
parallel_chain.get_graph().print_ascii()

### RunnablePassthrough
Serves as a simple utility for passing data through without modification. It is useful in a pipeline or chain of operations when you want to retain certain intermediate results or bypass specific steps without applying transformations.

In [ ]:
from operator import itemgetter

In [ ]:
# "mood" and "prompt" are passed as input to "tweet_chain"
# RunnablePassthrough() makes both of these passes through unchanged
# itemgetter extracts the value of mood parameter and assigned it to "mood"

tweet_chain = RunnableParallel(
    {
        "mood": RunnablePassthrough() | itemgetter("mood"),
        "tweet": tweet_generator
    }
) | tweet_fixer

In [ ]:
tweet_chain.invoke(
    {
        "prompt": "Langchain releases LangGraph for building stateful, multi-action applications. https://langchain.com",
        "mood": "sarcastic"
    }
)

### Batch
Using this method can significantly improve performance when needing to process multiple independent inputs as the processing can be done in parallel instead of sequentially.

In [ ]:
%time

response = parallel_chain.batch([{"topic": "bears"}, {"topic": "cats"}])

# the time measured is too tiny to tell the difference between parallel or batch chain

In [ ]:
response[0]['joke'].content

In [ ]:
response[0]['short_poem'].content

In [ ]:
response[1]['joke'].content

In [ ]:
response[1]['short_poem'].content

### Stream
Streaming enhances the responsiveness of application by displaying the output progressively, even before a complete response is ready. Due the latency of LLMs response generation, streaming improves user experience (UX).

In [ ]:
joke_str = ""
short_poem_str = ""

for s in parallel_chain.stream({"topic" : "dog"}):
    if 'joke' in s:
        joke_str = joke_str + s['joke'].content + " | "
    elif 'short_poem' in s:
        short_poem_str = short_poem_str + s['short_poem'].content + " | "

print(joke_str)
print('-'*10)
print(short_poem_str)